## Library importing

In [1]:
from langchain.tools import BaseTool, StructuredTool, tool
import os
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import PromptTemplate

import random


In [6]:
random.randint(0,5)

2

## LLM model declaration

In [4]:
GROQ_API_KEY = "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"

os.environ["GROQ_API_KEY"] = GROQ_API_KEY
llm = ChatGroq(temperature=0, model_name="llama-3.1-8b-instant")

In [31]:
from langchain_core.tools import tool

@tool
def multiply_two_number(a: int,b:int) -> int:
    """To multiply two given number such as a and b and give the result
    """
    return a*b

@tool
def random_numer_generator() -> int:
    """Generating random number from 0 to 5"""
    return random.randint(0,5)


tool_list = [multiply_two_number, random_numer_generator]

In [8]:
llm_tools = llm.bind_tools(tools)

In [15]:
llm_tools.invoke("what is 2 * 3? can you generate some number from 1 to 5 ")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5eqb', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply_two_number'}, 'type': 'function'}, {'id': 'call_fw5m', 'function': {'arguments': '{}', 'name': 'random_numer_generator'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 1013, 'total_tokens': 1126, 'completion_time': 0.150666667, 'prompt_time': 0.32230261, 'queue_time': None, 'total_time': 0.472969277}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bfd442b4-d536-4ec6-8dce-2d9fdbb428dc-0', tool_calls=[{'name': 'multiply_two_number', 'args': {'a': 2, 'b': 3}, 'id': 'call_5eqb', 'type': 'tool_call'}, {'name': 'random_numer_generator', 'args': {}, 'id': 'call_fw5m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1013, 'output_tokens': 113, 'total_tokens': 1126})

In [10]:
from langchain_core.tools import tool

@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city."""
    return "Sunny, 22°C"

@tool
def outdoor_seating_availability(city: str) -> str:
    """Check if outdoor seating is available at a specified restaurant in a given city."""
    return "Outdoor seating is available."


tools = [fake_weather_api, outdoor_seating_availability]

In [12]:
llm_with_tools = llm.bind_tools(tools)

result = llm_with_tools.invoke("How will the weather be in munich today? I would like to eat outside if possible")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zcsd', 'function': {'arguments': '{"city":"munich"}', 'name': 'fake_weather_api'}, 'type': 'function'}, {'id': 'call_fmhw', 'function': {'arguments': '{"city":"munich"}', 'name': 'outdoor_seating_availability'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 120, 'prompt_tokens': 1026, 'total_tokens': 1146, 'completion_time': 0.16, 'prompt_time': 0.300571946, 'queue_time': None, 'total_time': 0.46057194599999995}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e951f795-a33c-4c94-bbbc-d8fdea31ea25-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'city': 'munich'}, 'id': 'call_zcsd', 'type': 'tool_call'}, {'name': 'outdoor_seating_availability', 'args': {'city': 'munich'}, 'id': 'call_fmhw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1026, 'output_tokens': 120, 'total_tokens': 1146})

## Agent creator

In [34]:
from langchain.agents import create_tool_calling_agent
from langchain import hub
from langchain.agents import AgentExecutor

In [28]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

f:\set up\Anacoda\installation\envs\langchain-env\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [35]:

agent = create_tool_calling_agent(llm, tool_list, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

f:\set up\Anacoda\installation\envs\langchain-env\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


==> {'actions': [ToolAgentAction(tool='multiply_two_number', tool_input={'a': 2, 'b': 3}, log="\nInvoking: `multiply_two_number` with `{'a': 2, 'b': 3}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_2023', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply_two_number'}, 'type': 'function'}, {'id': 'call_j4gh', 'function': {'arguments': '{}', 'name': 'random_numer_generator'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-49fc4524-ee78-4cec-ac77-89570db07f5c', tool_calls=[{'name': 'multiply_two_number', 'args': {'a': 2, 'b': 3}, 'id': 'call_2023', 'type': 'tool_call'}, {'name': 'random_numer_generator', 'args': {}, 'id': 'call_j4gh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1022, 'output_tokens': 113, 'total_tokens': 1135}, tool_call_chunks=[{'name': 'multiply_two_number', 'args': '{"a":2,"b":3}', 'id': 'call_2023', 'index': None, 'type': 'tool_call_chunk'}, {'n

f:\set up\Anacoda\installation\envs\langchain-env\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


==> {'actions': [ToolAgentAction(tool='multiply_two_number', tool_input={'a': 6, 'b': 0}, log="\nInvoking: `multiply_two_number` with `{'a': 6, 'b': 0}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_89dw', 'function': {'arguments': '{"a":6,"b":0}', 'name': 'multiply_two_number'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cb1cf4b5-8dbf-4a81-a212-8a860bec36d4', tool_calls=[{'name': 'multiply_two_number', 'args': {'a': 6, 'b': 0}, 'id': 'call_89dw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1144, 'output_tokens': 41, 'total_tokens': 1185}, tool_call_chunks=[{'name': 'multiply_two_number', 'args': '{"a":6,"b":0}', 'id': 'call_89dw', 'index': None, 'type': 'tool_call_chunk'}])], tool_call_id='call_89dw')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_89dw', 'function': {'arguments': '{"a":6,"b":0}', 'name': 'multiply_two_number'}, 

f:\set up\Anacoda\installation\envs\langchain-env\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
